# Parsing documents for higher education Quality Assurance

### Brief Introduction to Higher Education Quality Assurance in Germany

Accreditation is a requirement of the German higher education system by which study programs and higher education institutions (HEIs) regularly undergo quality checks through external specialised agencies:

* the objective is to to ensure conformity with state-level higher education regulations. 
* these regulations include formal, objective requirements, pertaining for example to:

    - the length of degree programs
    - the degree types which can be awarded
    - the structure of the degree programs
    - the proper usage of the European Credit Transfer System (ECTS)
    - transparency of information
    - cooperation with other institutions
    
An example of one such regulation:

    "The standard periods of study for full-time study are six, seven or eight semesters for bachelor's programs and four, three or two semesters for master's programs. For bachelor's degree programs, the standard period of study for full-time study is at least three years. For consecutive degree programs, the total standard period of full-time study is five years (ten semesters)." (Source: Musterrechtsverordnung, Stiftung Akkreditierungsrat)

#### Current quality assurance procedure
To determine whether or not these formal criteria are met, auditors review the official documentation provided by the higher education institution. This documentation typically includes the syllabus, the examination regulations (_Prüfungsordnung_), the Module Catalog (_Modulhandbuch_) and a variety of other documents. Subsequently, auditors summarize their findings in the official Accreditation Report, which is submitted to the German Accreditation Board.

Essentially, this part of the procedure is a simple search and check exercise which, however, due to the often vast amounts of documentation provided, can be tedious to complete. Formal HEI documentation is seldomly structured in a manner conducive to auditing, and can significantly differ in terms of structure and content from one institution to the next. 

#### Project Objective

The goal of this project is therefore to parse formal HEI documents, check them for content relevant to the official formal criteria, and to - with the help of a Large Language Model (LLM) - summarize the findings in a manner so that they can be copy-pasted into the Accreditation Report. At the same time, we want to determine whether this is a viable approach in terms of costs.

While there are presumably multiple use-cases for LLMs in the context of accreditation, the one described above may be among the simplest ones to achieve, also because data samples in the form of formal documents published on German HEI websites are readily available. 

### Toolkit

the tools used will include:

* a PDF reader: almost all documentation for accreditation procedures is submitted in PDF format
* Langchain: a library which simplifies working with LLMs. It includes:
    - OpenAI Embeddings: fast embeddings API which charges a small amount per submitted token.
    - Huggingface Embeddings: for comparison we are also testing this free and high quality embeddings tool, although it takes up to 30x longer when run on a regular CPU.
* OpenAI GPT API: for accessing the GPT LLM 
* tiktoken: for converting text to tokens in order to estimate costs prior to embedding.
* Streamlit: a library for creating a simple user interface.


## Import libraries

In [1]:
from PyPDF2 import PdfReader
import openai
import json
from dotenv import load_dotenv
import tiktoken

# import langchain tools
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS                # Facebook AI Similarity Search - vectors are stored on machine - will be deleted once application is closed
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQA, LLMChain, HypotheticalDocumentEmbedder
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

C:\Users\Arne\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Arne\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Arne\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Create functions

### PDF text extraction functions

#### PDFMiner 
docs: https://github.com/euske/pdfminer/blob/master/pdfminer/pdfpage.py

In [2]:
# use PDF Miner
import io 
from pdfminer.converter import TextConverter 
from pdfminer.pdfinterp import PDFPageInterpreter 
from pdfminer.pdfinterp import PDFResourceManager 
from pdfminer.pdfpage import PDFPage 


def extract_text_by_page(pdf_path): 

    with open(pdf_path, 'rb') as fh: 
        
        for page in PDFPage.get_pages(fh, 
                                    caching=True, 
                                    check_extractable=True): 
            
            resource_manager = PDFResourceManager() 

            fake_file_handle = io.StringIO() 
            
            converter = TextConverter(resource_manager, 
                                    fake_file_handle) 
            
            page_interpreter = PDFPageInterpreter(resource_manager, 
                                                converter) 
            
            page_interpreter.process_page(page) 
            text = fake_file_handle.getvalue() 
            
            yield text 
            
            # close open handles 
            converter.close() 
            fake_file_handle.close() 
        

In [3]:
# text extracted via PDFminer still has hyphens from line-breaks, therefore we create another function

import re

def replace_hyphens(text):
    pattern = r'([a-z])-([a-z])'  # Pattern to match 'lowercase letter - lowercase letter'
    replacement = r'\1\2'         # Replacement pattern is equivalent to ''
    
    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text)
    
    # Iterate over the matches and replace the hyphen-separated lowercase letters
    for match in matches:
        text = text.replace(f'{match[0]}-{match[1]}', f'{match[0]}{match[1]}')
            
    return text

In [4]:
# split text into chunks
def get_text_chunks(text):
    
    text_splitter = RecursiveCharacterTextSplitter(
        #separator= "\n",
        chunk_size= 1000,
        chunk_overlap= 200,
        length_function= len
    )
    chunks= text_splitter.split_text(text)
    
    return chunks

In [5]:
# get number of tokens which will be submitted for embedding as well as price
def get_nr_of_tokens_and_price(chunks, price_per_1k_tokens):
    
    '''takes as arguments chunks created via previous function as well as price which can be researched on OpenAI website
    (https://openai.com/pricing)'''
    
    nr_tokens = 0
    
    for chunk in chunks:
        enc = tiktoken.get_encoding("p50k_base")
        chunk_tokens = enc.encode(chunk)
        nr_tokens += len(chunk_tokens)
        
    price = round((nr_tokens / 1000) * price_per_1k_tokens, 4)
        
    return nr_tokens, price

### Functions for creating Vector Stores

In [6]:
# create normal and Hypothetical Document Embedding (HyDE) vector stores
def get_vectorstore(text_chunks, create_hyde_store= True):
    
    # setup normal vector store
#     if response == '1':
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(text_chunks, embedding= embeddings) 
        
        # setup HyDE vector store
    if create_hyde_store:
        llm = OpenAI(temperature= 0)
        embeddings_hyde = HypotheticalDocumentEmbedder.from_llm(llm, embeddings, "web_search")
        vectorstore_hyde = FAISS.from_texts(text_chunks, embedding= embeddings_hyde) 
    
    print('EMBEDDING COMPLETED!')
    
    return vectorstore, vectorstore_hyde
    

In [7]:
def check_embeddings_with_user():
    
    # get price estimate
    price_per_1k_tokens = 0.0001
    nr_tokens, price = get_nr_of_tokens_and_price(text_chunks, price_per_1k_tokens)

    # communicate price of embedding and check with user if they want to proceed
    response = input(
        f'''The submitted text files have a length of {len(cleaned_text)} characters equivalent to {nr_tokens} tokens. 

    We can quickly embed these tokens for a total price of US$ {price} (rounded to 4 decimals). 
    Type '1' if you want to proceed with paid embedding. 
    Typing anything else will result in no action.\n''')

    if response == '1':
        create_store = True
        hyde_response = input(
    '''\nType "1" again if you also want a HyDE vector store (NOTE: this will double the embedding cost)\nTyping anything else will result in only a normal vector store being created\n\n''')

        if hyde_response == '1':
            create_hyde = True
        else:
            create_hyde = False
    else:
        create_store = False
    
    return create_store, create_hyde

### Functions for Key Word Identification

In [8]:
def count_key_words(text, word):
    word_count = text.lower().count(word)
    return word_count

In [9]:
def get_preferred_terms(text, term_dict= {
    "fin_proj_terms" : ["abschlussarbeit", "bachelorarbeit", "masterarbeit"],
    "creditpoint_terms" : ["kreditpunkte", "leistungspunkte", "ects-punkte", "ects punkte", " lp "],
    "degree_level": ["bachelorstudiengang", "masterstudiengang"],
    "degree_designation" : ["hochschulgrad", "abschlussgrad", "akademische Grad"]}
                          ):
    
    preferred_terms_dict = {}

    for x in term_dict:
        # get list of terms for category, e.g. "fin_proj_terms"
        term_list = term_dict[x]

        # get word count for each term in the list and add to new dict
        term_count_dict = {word:count_key_words(text, word) for word in term_list}

        # get term with highest occurence
        max_key = max(term_count_dict, key= term_count_dict.get)
    #     max_value = max(term_count_dict.values())
    
        preferred_terms_dict[x] = max_key
    #     preferred_terms_dict[x] = {max_key: max_value}

    return preferred_terms_dict

## PDF Upload and Text Split

In [10]:
# Load environment variables
load_dotenv()

file_path = '/Users/Arne/Downloads/Prüfungsordnungen/KIT_Ba_Informatik.pdf'

# extract individual pages
doc = extract_text_by_page(file_path)
pages = [page for page in doc]

# besides replacing hyphens we replace a string automatically added by PDFminer
cleaned_pages = [replace_hyphens(page).replace("\x0c", "") for page in pages]

cleaned_text = ''.join(cleaned_pages)

# get text chunks
text_chunks = get_text_chunks(cleaned_text)


In [11]:
print(text_chunks[0])

Amtliche Bekanntmachung     Die Forschungsuniversität in der Helmholtz-Gemeinschaft 2022  Ausgegeben Karlsruhe, den 10. Mai 2022 Nr. 34             I n h a l t Seite    Studien- und Prüfungsordnung des Karlsruher Instituts 215  für Technologie (KIT) für den Bachelorstudiengang  Informatik                   215   Studien- und Prüfungsordnung des Karlsruher Instituts für Technologie (KIT) für den Bachelorstudiengang Informatik  vom 10. Mai 2022  Aufgrund von § 10 Absatz 2 Ziff. 4 und § 20 Absatz 2 des Gesetzes über das Karlsruher Institut für Technologie (KIT-Gesetz - KITG) in der Fassung vom 14. Juli 2009 (GBl. S. 317 f), zuletzt geändert durch Artikel 2 des Gesetzes zur Änderung des Landeshochschulgesetzes und anderer Gesetze vom 26. Oktober 2021 (GBL. S. 941), und § 32 Absatz 3 Satz 1 des Gesetzes über die Hochschulen in Baden-Württemberg (Landeshochschulgesetz - LHG) in der Fassung vom 1. Januar 2005 zuletzt geändert durch Artikel 7 der Zehnten Verordnung des Innenministeriums


## Setup Vector Stores

In [12]:
create_store, create_hyde = check_embeddings_with_user()
print("Returning ", create_store, create_hyde)

if create_store:
    vector_store, vector_store_hyde = get_vectorstore(text_chunks, create_hyde_store= create_hyde)
else:
    exit()

The submitted text files have a length of 51608 characters equivalent to 25220 tokens. 

    We can quickly embed these tokens for a total price of US$ 0.0025 (rounded to 4 decimals). 
    Type '1' if you want to proceed with paid embedding. 
    Typing anything else will result in no action.
1

Type "1" again if you also want a HyDE vector store (NOTE: this will double the embedding cost)
Typing anything else will result in only a normal vector store being created

1
Returning  True True
EMBEDDING COMPLETED!


## Identify key terms

The document retrieval chain is sensitive to terminology - small differences in the submitted prompts, for example using the abbreviation "LP" for "Leistungspunkte" (or in English: using "CP" to refer to "credit points"), can generate very different results. We therefore first parse the text to identify the HEI-specific terminology for areas that are key to the analysis, in order to optimize the prompts.

In [21]:
# the following dict can be edited
term_dict = {"fin_proj_terms" : ["abschlussarbeit", "bachelorarbeit", "masterarbeit", "doktorarbeit"],
             "creditpoint_terms" : ["kreditpunkt", "leistungspunkt", "ects-punkt", "ects punkt", " lp "],
             "degree_level": ["bachelorstudiengang", "masterstudiengang", "phd"],
             "degree_designation" : ["hochschulgrad", "abschlussgrad", "akademische Grad"]
            }

preferred_terms = get_preferred_terms(cleaned_text, term_dict) # term_dict is an optional argument (default is included in function)

preferred_terms

{'fin_proj_terms': 'bachelorarbeit',
 'creditpoint_terms': 'leistungspunkt',
 'degree_level': 'bachelorstudiengang',
 'degree_designation': 'hochschulgrad'}

### Adjust prompts

In [22]:
if "bachelor" in preferred_terms['degree_level']:
    degree= "bachelor"
elif "master" in preferred_terms['degree_level']:
    degree= "master"
else:
    degree= ""

prompts = ["Was ist die Regelstudienzeit des Studiengangs? Welche Ausnahmen gibt es?", 
           f"Wieviele {preferred_terms['creditpoint_terms']}e umfasst der Studiengang?",
           f'''Was ist der Umfang der {preferred_terms['fin_proj_terms']}, bzw. wieviele {preferred_terms['creditpoint_terms']}e umfasst die {preferred_terms['fin_proj_terms']}? welchen umfang hat das modul '{preferred_terms["fin_proj_terms"]}'?''',
           f"Welcher akademische Grad wird verliehen? (z.B. '{degree} of Science', '{degree} of Arts' oder '{degree} of Engineering')",
           "Wie sind die Zugangsvoraussetzungen für den Studiengang?",
           f"Wieviele Arbeitsstunden sind in einem {preferred_terms['creditpoint_terms']} enthalten?",
           "In wiefern werden ein Diploma Supplement und ein Zeugnis und ein Transcript of Records ausgestellt?"
          ]

if degree == 'master':
    prompts.append("Ist der Masterstudiengang konsekutiv oder weiterbildend?")

prompts

['Was ist die Regelstudienzeit des Studiengangs? Welche Ausnahmen gibt es?',
 'Wieviele leistungspunkte umfasst der Studiengang?',
 "Was ist der Umfang der bachelorarbeit, bzw. wieviele leistungspunkte umfasst die bachelorarbeit? welchen umfang hat das modul 'bachelorarbeit'?",
 "Welcher akademische Grad wird verliehen? (z.B. 'bachelor of Science', 'bachelor of Arts' oder 'bachelor of Engineering')",
 'Wie sind die Zugangsvoraussetzungen für den Studiengang?',
 'Wieviele Arbeitsstunden sind in einem leistungspunkt enthalten?',
 'In wiefern werden ein Diploma Supplement und ein Zeugnis und ein Transcript of Records ausgestellt?']

## Setup Retrieval Chains
This sets up a chain which can take user queries as an input. The queries are embedded and similar documents are retrieved from the vector store. The LLM then formulates an answer based on the retrieved text chunks.

In the case of the HyDE chain, the user query and the LLM are used to create a Hypothetical answer to the query. For example:

- User query: "What did the president say about relations with Indonesia?"
- Hypothetical answer generated by LLM: "The president said that U.S. relations with Indonesia have grown closer and closer."

This hypothetical answer is then embedded and submitted to the vector store INSTEAD OF (or in addition to?) THE USER QUERY in order to retrieve relevant documents. This can result in a much better match.

In [ ]:
llm = OpenAI(temperature= 0.0)   # initialize LLM model
turbo_llm = ChatOpenAI(temperature= 0.0,
                       model_name='gpt-3.5-turbo')

                       
'''Notes on from_llm method: takes llm, prompt (see prompt template) and any kwargs as arguments'''

retrieval_chain = RetrievalQA.from_llm(
    llm = llm,
    retriever = vector_store.as_retriever(search_kwargs={"k": 3}),
    #memory = memory,
    return_source_documents= True
)


# # create retrieval chain with HyDE vectorstore
# retrieval_chain_hyde = RetrievalQA.from_llm(
#     llm = llm,
#     retriever = vector_store_hyde.as_retriever(search_kwargs={"k": 1}),
#     return_source_documents= True
# )

### Comparison of queries

The sample questions and answers below provide an indication of the sensitivity of the model to terminology used in the query. The relevant text in the document reads: "Der Umfang der Bachelorarbeit entspricht 12 Leistungspunkten." 

While the first query results in a correct answer, neither the second nor third query are unable to detect it. It should be noted that the first query contains 4 of the words that are contained in the relevant string, in the same order (...der Umfang der Bachelorarbeit)

This indicates that, when searching for relevant documents, query wording is extremely important. In fact, even capitalization appears to play a role, as it changes the response.

Possible measures to improve performance:
- use longer questions which through the usage of more words may result in embeddings with a higher proximity
- lowercase all words before embedding
- include metadata in both the documents and the queries
- use different embedding model

In [ ]:
# Comparison of 3 queries with slight differences in wording
queries = ['was ist der umfang der bachelorarbeit?',
           'wieviele leistungspunkte umfängt die bachelorarbeit?',
           'wieviele leistungspunkten entspricht die bachelorarbeit?',
          "Was ist der Umfang der bachelorarbeit, bzw. wieviele leistungspunkte umfasst die bachelorarbeit? welchen umfang hat das modul 'bachelorarbeit'?"]

# for query in queries:
#     result = retrieval_chain({"query": query})
#     print(result)

In [ ]:
# Use 'apply' method to get all results in a list
retrieval_chain.apply(queries)

In [ ]:
for query in prompts:
    result = retrieval_chain({"query": query})
    print(result)

In [ ]:
query = "In wiefern werden ein Diploma Supplement und ein Zeugnis und ein Transcript of Records ausgestellt?"
retrieval_chain({"query": query})

In [ ]:
query = '''In wiefern erfüllt der Studiengang folgendes Kriterium: 

(1) 1 Im System gestufter Studiengänge ist der Bachelorabschluss der erste berufsqualifizierende Regelabschluss eines Hochschulstudiums; der Masterabschluss stellt einen weiteren berufsqualifizierenden Hochschulabschluss dar. 2Grundständige Studiengänge, die unmittelbar zu einem Masterabschluss führen, sind mit Ausnahme der in Absatz 3 genannten Studiengänge ausgeschlossen. (2) 1Die Regelstudienzeiten für ein Vollzeitstudium betragen sechs, sieben oder acht Semester bei den Bachelorstudiengängen und vier, drei oder zwei Semester bei den Masterstudiengängen. 2 Im Bachelorstudium beträgt die Regelstudienzeit im Vollzeitstudium mindestens drei Jahre. 3Bei konsekutiven Studiengängen beträgt die Gesamtregelstudienzeit im Vollzeitstudium fünf Jahre (zehn Semester). 4Wenn das Landesrecht dies vorsieht, sind kürzere und längere Regelstudienzeiten bei entsprechender studienorganisatorischer - 3 Gestaltung ausnahmsweise möglich, um den Studierenden eine individuelle Lernbiografie, insbesondere durch Teilzeit-, Fern-, berufsbegleitendes oder duales Studium sowie berufspraktische Semester, zu ermöglichen. 5Abweichend von Satz 3 können in den künstlerischen Kernfächern an Kunstund Musikhochschulen nach näherer Bestimmung des Landesrechts konsekutive Bachelor- und Masterstudiengänge auch mit einer Gesamtregelstudienzeit von sechs Jahren eingerichtet werden. (3) Theologische Studiengänge, die für das Pfarramt, das Priesteramt und den Beruf der Pastoralreferentin oder des Pastoralreferenten qualifizieren („Theologisches Vollstudium“), müssen nicht gestuft sein und können eine Regelstudienzeit von zehn Semestern aufweisen.
'''
retrieval_chain({"query": query})

### HyDE with multi-document generation

In [ ]:
multi_llm = OpenAI(n=2, best_of=2)

embeddings_multi = HypotheticalDocumentEmbedder.from_llm(
    multi_llm, base_embeddings, "web_search"
)

vectorstore_multi = FAISS.from_texts(text_chunks, embedding= embeddings_multi) 

In [ ]:
# create retrieval chain with HyDE vectorstore
retrieval_chain_hyde_m = RetrievalQA.from_llm(
    llm = llm,
    retriever = vectorstore_multi.as_retriever(search_kwargs={"k": 1}),
    return_source_documents= True
)

In [ ]:
query = 'was ist der umfang der bachelorarbeit, bzw. wieviele leistungspunkte umfasst die bachelorarbeit?'
retrieval_chain_hyde_m({"query": query})

### Notes
- HyDE seems to work much better - much more flexibility in formulating questions. 3 / 3 correct answers instead of 1/3
- but also much slower, probably about 10-15x slower. Up to 1 minute to answer 3 questions
- does not perform well when mixing questions which might be covered in multiple docs

#### To-Dos
- Use other vector stores? Chroma??
- Unit tests (save for PyCharm)
- Look at HyDE docs - appears doc is submitted instead of user query, Can I see what kind of document is generated? HyDE can perhaps be used for a chatbot application (see docs: https://github.com/hwchase17/langchain/blob/8502117f62fc4caa53d504ccc9d4e6a512006e7f/langchain/chains/hyde/base.py#L20)
- Need to setup QA: proportion of chatbot answers that are correct?
- Setup columns in Streamlit which display docs and proposed answers
- Annotation - retrieval of relevant string snippets?
- Extracting entities? https://python.langchain.com/docs/modules/chains/additional/extraction
- Add Metadata to docs: https://python.langchain.com/docs/modules/chains/popular/vector_db_qa#return-source-documents:~:text=Return%20Source%20Documents
- Adapt to cheapest LLM model (ChatOpenAI model_name = gpt-3.5-turbo)

### Prompt

In [ ]:
def create_prompt(question):
    prompt_template_de = f'''Du bist ein Audit-Assistent für Qualitätssicherung im Hochschulwesen. 
    Deine Aufgabe ist es, Fragen über die Inhalte von Hochschulunterlagen wahrheitsgemäß zu beantworten. 
    Wenn die Unterlagen über keine Inhalte verfügen, die für die gestellte Frage relevant sind, teilst du dies mit. 
    Dein Ton ist formal und professionell. 
    
    Frage: "Welche Informationen gibt es bezüglich der Vergabe eines Diploma Supplements?"
    Antwort: "Gemäß der Prüfungsordnung erhalten die Studierenden beim Abschluss des Studiums ein Diploma Supplement auf Deutsch und Englisch wie auch ein Transcript of Records"

    Frage: "In wiefern verfügen die Studiengänge über Kooperationen mit nicht-hochschulischen Einrichtungen?"
    Antwort: "Aus den zur Verfügung stehenden Unterlagen ist es nicht ersichtlich, in wiefern Kooperationen mit nicht-hochschulischen Einrichtungen bestehen.

    Frage: "Wenn es sich um einen Masterstudiengang handelt, ist er konsekutiv oder weiterbildend?"
    Antwort: "Es handelt sich um einen Bachelorstudiengang, daher ist dieses Kriterium für diesen Studiengang nicht relevant."

    Frage: {question}
    Antwort: '''
    
    return prompt_template_de

### Setup loop

In [25]:
import json
def load_json(file_path):
    file = open(file_path, encoding="utf-8")   # specifying encoding necessary to display German characters
    criteria_sets = json.load(file)
    return criteria_sets

In [58]:
file_path = 'acc_checker/criteria_sets_simple.json'
criteria_sets_dict = load_json(file_path)

## Functions for getting prompts from and storing responses and sources to dict

In [101]:
def get_llm_response_with_sources(retrieval_chain, prompt):
    response = retrieval_chain({"query": prompt})
    return response

In [90]:
# source documents must be list containing <class 'langchain.schema.Document'> objects
def combine_doc_strings(source_documents):
    if source_documents is None:
        raise Exception("No source document returned!")
    else:
        source_str = ""
        for d in source_documents:
            source_str += d.page_content
        return source_str

In [113]:
def get_and_store_llm_response_and_source_docs(crit_dict, retrieval_chain):
    result = get_llm_response_with_sources(retrieval_chain, crit_dict["prompt"])
    crit_dict["response"] = result["result"]
    crit_dict["source"] = combine_doc_strings(result["source_documents"])

In [114]:
# takes criteria set dict and langchain retrieval chain as arguments
# returns list which is a version of the original criteria list, expanded to include LLM responses and retrieved source docs
def get_and_store_all_llm_responses_and_source_docs(criteria_set_dict, retrieval_chain):
    criteria_and_response_list = criteria_set_dict['criteria_sets'][0]['criteria']
    for c in criteria_and_response_list:
        if c["subcriteria"]:
            for s in c["subcriteria"]:
                if s["prompt"]:
                    get_and_store_llm_response_and_source_docs(s, retrieval_chain)
                else:
                    raise Exception(f"Missing prompt for criterion: {c['name']}, subcriterion {s['name']}")
        elif c["prompt"]:
            get_and_store_llm_response_and_source_docs(c, retrieval_chain)
        else:
            raise Exception(f"Missing prompt for criterion: {c['name']}")
            
    return criteria_and_response_list

In [125]:
class AnalysisExecutor:
    
    def __init__(self, criteria_set_dict, vector_store):
        self.criteria_set = criteria_set_dict
        self.vector_store = vector_store
        self.retrieval_chain = self.get_retrieval_chain(vector_store)
        self.answer_list = self.get_and_store_all_llm_responses_and_source_docs(criteria_set_dict, self.retrieval_chain)
        
    def get_retrieval_chain(self, vector_store):
        llm = OpenAI(temperature= 0.0)   # initialize LLM model
#         turbo_llm = ChatOpenAI(temperature= 0.0, model_name='gpt-3.5-turbo')
        retrieval_chain = RetrievalQA.from_llm(
            llm = llm,
            retriever = vector_store.as_retriever(search_kwargs={"k": 3}),
            #memory = memory,
            return_source_documents= True)
        return retrieval_chain
    
    
    def get_llm_response_with_sources(self, retrieval_chain, prompt):
        response = retrieval_chain({"query": prompt})
        return response
    
    
    # source documents must be list containing <class 'langchain.schema.Document'> objects
    def combine_doc_strings(self, source_documents):
        if source_documents is None:
            raise Exception("No source document returned!")
        else:
            source_str = ""
            for d in source_documents:
                source_str += d.page_content
            return source_str
    
    
    def get_and_store_llm_response_and_source_docs(self, crit_dict, retrieval_chain):
        result = get_llm_response_with_sources(retrieval_chain, crit_dict["prompt"])
        crit_dict["response"] = result["result"]
        crit_dict["source"] = combine_doc_strings(result["source_documents"])
        
        
    # takes criteria set dict and langchain retrieval chain as arguments
    # returns list which is a version of the original criteria list, expanded to include LLM responses and retrieved source docs
    def get_and_store_all_llm_responses_and_source_docs(self, criteria_set_dict, retrieval_chain):
        criteria_and_response_list = criteria_set_dict['criteria_sets'][0]['criteria']
        for c in criteria_and_response_list:
            if c["subcriteria"]:
                for s in c["subcriteria"]:
                    if s["prompt"]:
                        get_and_store_llm_response_and_source_docs(s, retrieval_chain)
                    else:
                        raise Exception(f"Missing prompt for criterion: {c['name']}, subcriterion {s['name']}")
            elif c["prompt"]:
                get_and_store_llm_response_and_source_docs(c, retrieval_chain)
            else:
                raise Exception(f"Missing prompt for criterion: {c['name']}")

        return criteria_and_response_list

In [126]:
analyzer = AnalysisExecutor(criteria_sets_dict, vector_store)
analyzer.answer_list

[{'name': '§ 3 Studienstruktur und Studiendauer',
  'name_short': '§ 3',
  'position': 1,
  'text': '(1) 1 Im System gestufter Studiengänge ist der Bachelorabschluss der erste berufsqualifizierende Regelabschluss eines Hochschulstudiums; der Masterabschluss stellt einen weiteren berufsqualifizierenden Hochschulabschluss dar. 2Grundständige Studiengänge, die unmittelbar zu einem Masterabschluss führen, sind mit Ausnahme der in Absatz 3 genannten Studiengänge ausgeschlossen. (2) 1Die Regelstudienzeiten für ein Vollzeitstudium betragen sechs, sieben oder acht Semester bei den Bachelorstudiengängen und vier, drei oder zwei Semester bei den Masterstudiengängen. 2 Im Bachelorstudium beträgt die Regelstudienzeit im Vollzeitstudium mindestens drei Jahre. 3Bei konsekutiven Studiengängen beträgt die Gesamtregelstudienzeit im Vollzeitstudium fünf Jahre (zehn Semester). 4Wenn das Landesrecht dies vorsieht, sind kürzere und längere Regelstudienzeiten bei entsprechender studienorganisatorischer Gest

In [118]:
answer_list = get_and_store_responses(criteria_sets_dict, retrieval_chain)

In [119]:
answer_list

[{'name': '§ 3 Studienstruktur und Studiendauer',
  'name_short': '§ 3',
  'position': 1,
  'text': '(1) 1 Im System gestufter Studiengänge ist der Bachelorabschluss der erste berufsqualifizierende Regelabschluss eines Hochschulstudiums; der Masterabschluss stellt einen weiteren berufsqualifizierenden Hochschulabschluss dar. 2Grundständige Studiengänge, die unmittelbar zu einem Masterabschluss führen, sind mit Ausnahme der in Absatz 3 genannten Studiengänge ausgeschlossen. (2) 1Die Regelstudienzeiten für ein Vollzeitstudium betragen sechs, sieben oder acht Semester bei den Bachelorstudiengängen und vier, drei oder zwei Semester bei den Masterstudiengängen. 2 Im Bachelorstudium beträgt die Regelstudienzeit im Vollzeitstudium mindestens drei Jahre. 3Bei konsekutiven Studiengängen beträgt die Gesamtregelstudienzeit im Vollzeitstudium fünf Jahre (zehn Semester). 4Wenn das Landesrecht dies vorsieht, sind kürzere und längere Regelstudienzeiten bei entsprechender studienorganisatorischer Gest

### Testing

In [ ]:
def get_answer_dict(criteria_set_dict):
    criteria_list = criteria_set_dict['criteria_sets'][0]['criteria']
    answer_dict_elements = []
    for c in criteria_list:
        if c["subcriteria"]:
            for s in c['subcriteria']:
                if s["prompt"]:
                    s["answer"]= None
                    answer_dict_elements.append()
                else:
                    raise Exception(f"Missing prompt for criterion: {c['name']}, subcriterion {s['name']}")
        elif c["prompt"]:
            prompts.append(c['prompt']) 
        else:
            raise Exception(f"Missing prompt for criterion: {c['name']}")
            
    return criteria_list, prompts

In [ ]:
def store_results_with_sources_in_dict(criteria_list, responses):
    
    # seperate results from original criteria by storing them in a new list
    criteria_and_results_list = criteria_list.copy()
    for c in criteria_and_results_list:
        
        # if there are subcriteria, we loop through these and add the responses and source docs
        if c["subcriteria"]:
            
            
            for s, r in zip(c['subcriteria'], responses):
                s["result"] = r["result"]
                s["source"]= combine_doc_strings(r["source_documents"])
                
        # else we loop through the criteria
        elif c["prompt"]:        
            c["result"] = r["result"]
            
                if s["prompt"]:
                    prompts.append(s['prompt'])
                else:
                    raise Exception(f"Missing prompt for criterion: {c['name']}, subcriterion {s['name']}")
        
            prompts.append(c['prompt']) 
        else:
            raise Exception(f"Missing prompt for criterion: {c['name']}")
        
        # expects that subcriteria and response are of the same length
        for s, r in zip(c['subcriteria'], responses):
            s["result"] = r["result"]
            source_str = ""
            for d in r["source_documents"]:
                source_str += d.page_content
            s["source"]= source_str

In [69]:
criteria_list, prompts = get_criteria_list_and_prompts(criteria_sets_dict)
print("CRITERIA:", criteria_list)
print("PROMPTS:", prompts)

CRITERIA: [{'name': '§ 3 Studienstruktur und Studiendauer', 'name_short': '§ 3', 'position': 1, 'text': '(1) 1 Im System gestufter Studiengänge ist der Bachelorabschluss der erste berufsqualifizierende Regelabschluss eines Hochschulstudiums; der Masterabschluss stellt einen weiteren berufsqualifizierenden Hochschulabschluss dar. 2Grundständige Studiengänge, die unmittelbar zu einem Masterabschluss führen, sind mit Ausnahme der in Absatz 3 genannten Studiengänge ausgeschlossen. (2) 1Die Regelstudienzeiten für ein Vollzeitstudium betragen sechs, sieben oder acht Semester bei den Bachelorstudiengängen und vier, drei oder zwei Semester bei den Masterstudiengängen. 2 Im Bachelorstudium beträgt die Regelstudienzeit im Vollzeitstudium mindestens drei Jahre. 3Bei konsekutiven Studiengängen beträgt die Gesamtregelstudienzeit im Vollzeitstudium fünf Jahre (zehn Semester). 4Wenn das Landesrecht dies vorsieht, sind kürzere und längere Regelstudienzeiten bei entsprechender studienorganisatorischer 

In [70]:
responses = get_llm_responses_with_sources(retrieval_chain, prompts)
responses

[{'query': 'blablabla',
  'result': " I don't know.",
  'source_documents': [Document(page_content='stattfinden. (2) 1Die Bachelorarbeit kann von Hochschullehrer/innen und leitenden Wissenschaftler/innen gemäß § 14 Abs. 3 Ziff. 1 KITG in der Fassung vor Inkrafttreten des 2. KIT-WG vom 04. Februar 2021 sowie habilitierten Mitgliedern der KIT-Fakultät für Informatik vergeben werden. 2Darüber hinaus kann der Prüfungsausschuss weitere Prüfende gemäß § 18 Abs. 2 und 3 zur Vergabe des Themas berechtigen. 3Den Studierenden ist Gelegenheit zu geben, für das Thema Vorschläge zu machen. 4Soll die Bachelorarbeit außerhalb der KIT-Fakultät für Informatik angefertigt werden, so bedarf dies der Genehmigung durch den Prüfungsausschuss. 5Die Bachelorarbeit kann auch in Form einer Gruppenarbeit zugelassen werden, wenn der als Prüfungsleistung zu bewertende Beitrag der/des einzelnen Studierenden aufgrund objektiver Kriterien, die eine eindeutige Abgrenzung ermöglichen, deutlich unterscheidbar ist und di

In [77]:
criteria

[{'name': '§ 3 Studienstruktur und Studiendauer',
  'name_short': '§ 3',
  'position': 1,
  'text': '(1) 1 Im System gestufter Studiengänge ist der Bachelorabschluss der erste berufsqualifizierende Regelabschluss eines Hochschulstudiums; der Masterabschluss stellt einen weiteren berufsqualifizierenden Hochschulabschluss dar. 2Grundständige Studiengänge, die unmittelbar zu einem Masterabschluss führen, sind mit Ausnahme der in Absatz 3 genannten Studiengänge ausgeschlossen. (2) 1Die Regelstudienzeiten für ein Vollzeitstudium betragen sechs, sieben oder acht Semester bei den Bachelorstudiengängen und vier, drei oder zwei Semester bei den Masterstudiengängen. 2 Im Bachelorstudium beträgt die Regelstudienzeit im Vollzeitstudium mindestens drei Jahre. 3Bei konsekutiven Studiengängen beträgt die Gesamtregelstudienzeit im Vollzeitstudium fünf Jahre (zehn Semester). 4Wenn das Landesrecht dies vorsieht, sind kürzere und längere Regelstudienzeiten bei entsprechender studienorganisatorischer Gest

In [87]:
crit_list = [1, 2, 3, 4, 5, 6, 7, 8, 9]
answer_list = ["a", "b", "c", "d", "e", "f", "g"]
for c, a in zip(crit_list, answer_list):
    print(c, a)

1 a
2 b
3 c
4 d
5 e
6 f
7 g


In [88]:
criteria_and_response_list = criteria.copy()
for 

In [ ]:
#     # if there are subcriteria, we loop through these and add the responses and source docs
#     if c["subcriteria"]:


#         for s, r in zip(c['subcriteria'], responses):
#             s["result"] = r["result"]
#             s["source"]= combine_doc_strings(r["source_documents"])

#     # else we loop through the criteria
#     elif c["prompt"]:        
#         c["result"] = r["result"]

#             if s["prompt"]:
#                 prompts.append(s['prompt'])
#             else:
#                 raise Exception(f"Missing prompt for criterion: {c['name']}, subcriterion {s['name']}")

#         prompts.append(c['prompt']) 
#     else:
#         raise Exception(f"Missing prompt for criterion: {c['name']}")

#     # expects that subcriteria and response are of the same length
#     for s, r in zip(c['subcriteria'], responses):
#         s["result"] = r["result"]
#         source_str = ""
#         for d in r["source_documents"]:
#             source_str += d.page_content
#         s["source"]= source_str

[{'query': 'blablabla',
  'result': " I don't know.",
  'source_documents': [Document(page_content='stattfinden. (2) 1Die Bachelorarbeit kann von Hochschullehrer/innen und leitenden Wissenschaftler/innen gemäß § 14 Abs. 3 Ziff. 1 KITG in der Fassung vor Inkrafttreten des 2. KIT-WG vom 04. Februar 2021 sowie habilitierten Mitgliedern der KIT-Fakultät für Informatik vergeben werden. 2Darüber hinaus kann der Prüfungsausschuss weitere Prüfende gemäß § 18 Abs. 2 und 3 zur Vergabe des Themas berechtigen. 3Den Studierenden ist Gelegenheit zu geben, für das Thema Vorschläge zu machen. 4Soll die Bachelorarbeit außerhalb der KIT-Fakultät für Informatik angefertigt werden, so bedarf dies der Genehmigung durch den Prüfungsausschuss. 5Die Bachelorarbeit kann auch in Form einer Gruppenarbeit zugelassen werden, wenn der als Prüfungsleistung zu bewertende Beitrag der/des einzelnen Studierenden aufgrund objektiver Kriterien, die eine eindeutige Abgrenzung ermöglichen, deutlich unterscheidbar ist und di